In [7]:
import requests
import json
import pandas as pd

In [8]:
player_info_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2020-21&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='

In [9]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [10]:
response = requests.get(url=player_info_url, headers=headers).json()

In [11]:
response

{'resource': 'leaguedashplayerstats',
 'parameters': {'MeasureType': 'Base',
  'PerMode': 'PerGame',
  'PlusMinus': 'N',
  'PaceAdjust': 'N',
  'Rank': 'N',
  'LeagueID': '00',
  'Season': '2020-21',
  'SeasonType': 'Regular Season',
  'PORound': 0,
  'Outcome': None,
  'Location': None,
  'Month': 0,
  'SeasonSegment': None,
  'DateFrom': None,
  'DateTo': None,
  'OpponentTeamID': 0,
  'VsConference': None,
  'VsDivision': None,
  'TeamID': 0,
  'Conference': None,
  'Division': None,
  'GameSegment': None,
  'Period': 0,
  'ShotClockRange': None,
  'LastNGames': 0,
  'GameScope': None,
  'PlayerExperience': None,
  'PlayerPosition': None,
  'StarterBench': None,
  'DraftYear': None,
  'DraftPick': None,
  'College': None,
  'Country': None,
  'Height': None,
  'Weight': None,
  'TwoWay': 0},
 'resultSets': [{'name': 'LeagueDashPlayerStats',
   'headers': ['PLAYER_ID',
    'PLAYER_NAME',
    'NICKNAME',
    'TEAM_ID',
    'TEAM_ABBREVIATION',
    'AGE',
    'GP',
    'W',
    'L',
  

In [12]:
df = pd.json_normalize(response['resultSets'], record_path = 'rowSet') ##Grab the right selection from the json and puts it in rows
df.columns = pd.json_normalize(response['resultSets'], record_path = 'headers')[0].tolist() ##Takes header columns and adds it to the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 66 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PLAYER_ID             540 non-null    int64  
 1   PLAYER_NAME           540 non-null    object 
 2   NICKNAME              540 non-null    object 
 3   TEAM_ID               540 non-null    int64  
 4   TEAM_ABBREVIATION     540 non-null    object 
 5   AGE                   540 non-null    float64
 6   GP                    540 non-null    int64  
 7   W                     540 non-null    int64  
 8   L                     540 non-null    int64  
 9   W_PCT                 540 non-null    float64
 10  MIN                   540 non-null    float64
 11  FGM                   540 non-null    float64
 12  FGA                   540 non-null    float64
 13  FG_PCT                540 non-null    float64
 14  FG3M                  540 non-null    float64
 15  FG3A                  5

## Generate SQL Tables from Pandas DataFrame

In [13]:
def SQL_CREATE_STATEMENT_FROM_DATAFRAME(SOURCE, TARGET):

# SQL_CREATE_STATEMENT_FROM_DATAFRAME(SOURCE, TARGET)
# SOURCE: source dataframe
# TARGET: target table to be created in database

    sql_text = pd.io.sql.get_schema(SOURCE.reset_index(), TARGET) + ';'   
    return sql_text

def SQL_INSERT_STATEMENT_FROM_DATAFRAME(SOURCE, TARGET):
    sql_texts = []
    for index, row in SOURCE.iterrows():       
        sql_texts.append('INSERT INTO '+TARGET+' ('+ str(', '.join(SOURCE.columns))+ ') VALUES '+ str(tuple(row.values)) + ';')        
    return '\n\n'.join(sql_texts)

def write_To_SQL_Statements(dataframe, tableName, textFileName):
    # dataframe: pandas df to write to file
    # tableName: desired table name of sql table
    # textFileName: name of text file to write sql statements (may need an 'r' before text (example: r/'output.txt'))
    File = open(textFileName, "w+")
    File.write('DROP TABLE IF EXISTS ' + tableName + ';\n\n')
    File.write(SQL_CREATE_STATEMENT_FROM_DATAFRAME(dataframe, tableName))
    File.write('\n\n')
    File.write(SQL_INSERT_STATEMENT_FROM_DATAFRAME(dataframe, tableName))
    File.close()

In [14]:
File = open(r"test.txt", "w+")
File.write(SQL_CREATE_STATEMENT_FROM_DATAFRAME(df, 'players2021'))
File.write('\n\n')
File.write(SQL_INSERT_STATEMENT_FROM_DATAFRAME(df, 'players2021'))
File.close()

In [15]:
write_To_SQL_Statements(df, 'players2021', 'test.txt')